Just using this to write and easily test the code for the baseline model. Final implementation will be in a py script, so it can be run from command line using GPU.


# To do!
- create function to extract data to train model
- create function to output tags into appropriate format
- make sure model works
- train model
- submit results



In [4]:
import numpy as np
import torch
import TorchCRF
from torch import nn

#Creating the class for the baseline Model

class baselineModel(torch.nn.Module):

    def __init__(self, nWords, tags, dimEmbed, dimHidden):
        super().__init__()
        self.dimEmbed = dimEmbed
        self.dimHidden = dimHidden
        self.vocabSize = nWords
        self.tagSetSize = len(tags)
        self.tagSet = tags

        self.embed = nn.Embedding(nWords, dimEmbed)
        self.LSTM = nn.LSTM(dimEmbed, dimHidden, batch_first = True, bidirectional = True)
        self.linear = nn.Linear(dimHidden * 2, self.tagSetSize) 
        
        self.CRF = torchcrf.CRF(self.tagSetSize, batch_first = True)


        
    def forwardTrain(self, inputData):
        wordVectors = self.embed(inputData)
        out, _ = self.LSTM(wordVectors.view((inputData.shape[0], inputData.shape[1], self.dimEmbed)))
        out = self.linear(out)
        preds = self.CRF.decode(out)
        loss = self.CRF.forward(out, preds)
        return loss
        
        
    def forwardPred(self, inputData):
        wordVectors = self.embed(inputData)
        out, _ = self.LSTM(wordVectors.view((inputData.shape[0], inputData.shape[1], self.dimEmbed)))
        out = self.linear(out)
        preds = self.CRF.decode(out)
        return preds
